In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import json

# Load model
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
hf_token =HF_TOKEN # 🔒 Replace with your token

tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map={"": 0},  # ✅ Force everything on GPU 0
    quantization_config=quant_config,
    torch_dtype=torch.float16,
    token=hf_token,
    trust_remote_code=True
).eval()

def format_inst_prompt(resume_text):
    return f"""[INST] Convert the following resume into structured JSON with this format:
{{
  "summary": string,
  "skills": [string],
  "education": [{{"degree": string, "field": string, "institution": string, "year": string}}],
  "experience": [{{"job_title": string, "company": string, "start_date": string, "end_date": string, "description": string}}]
}}

Resume:
{resume_text}

Return JSON only. [/INST]
"""

def run_resume_to_json(resume_text):
    prompt = format_inst_prompt(resume_text)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=2048,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            do_sample=False
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    print("\n📤 Prompt:\n", prompt)
    print("\n📥 Raw Output:\n", decoded)

    # Try parsing
    try:
        start_idx = decoded.find("{")
        json_output = json.loads(decoded[start_idx:])
        print("\n✅ Parsed JSON:")
        print(json.dumps(json_output, indent=2))
        return json_output
    except Exception as e:
        print("\n❌ Failed to parse JSON:", str(e))
        return None

# 🔍 Example resume
sample_resume = """
John Doe

SUMMARY
Experienced software engineer with 10+ years...

PROFESSIONAL EXPERIENCE
Senior Software Engineer, ABC Corp
Built microservices in Java and deployed on Kubernetes.

EDUCATION
MS in Computer Science, XYZ University

SKILLS
Java, React, Spring Boot, SQL, Docker
"""

run_resume_to_json(sample_resume)
